In [1]:
# Now we are going to run a basic simulation
from copy import deepcopy as dc
from ase import Atoms
from pyiid.experiments.elasticscatter import ElasticScatter
from pyiid.calc.calc_1d import Calc1D
from pyiid.sim.nuts_hmc import NUTSCanonicalEnsemble
from ase.cluster import Octahedron
import matplotlib.pyplot as plt
from ase.visualize import view

In [2]:
atoms = Octahedron('Au', 2)

In [6]:
# We can examine the configuration we made
atoms.edit()

/home/christopher/mc/envs/sim_dev2/lib/python2.7/site-packages/gtk-2.0/gtk/__init__.py:127: RuntimeWarning: PyOS_InputHook is not available for interactive use of PyGTK
  set_interactive(1)


In [3]:
scat = ElasticScatter()
pdf = scat.get_pdf(atoms)
r = scat.get_r()

In [7]:
# Now lets dilate the atoms so that they don't match the pdf
atoms2 = dc(atoms)
atoms2.positions *= 1.03
pdf2 = scat.get_pdf(atoms2)
r = scat.get_r()

In [8]:
calc = Calc1D(
    target_data=pdf,  # The target or experimental data
    exp_function=scat.get_pdf,
    # The function which takes in atoms and produces
    # data like the experiment
    exp_grad_function=scat.get_grad_pdf,  # the function which produces the
    #  gradient of the calculated data
    conv=100,  # conversion from the unitless goodness of fit to eV
    potential='rw' # use the rw PES over chi squared
)

In [9]:
# Now we attach the calculator to our displaced atoms
atoms2.set_calculator(calc)
# Now we can get back the potential energy
print atoms2.get_potential_energy()
# And the forces
print atoms2.get_forces()

92.551192825
[[ 52.77273718  -0.          -0.        ]
 [ -0.          52.77273756  -0.        ]
 [ -0.          -0.          52.77273732]
 [ -0.          -0.         -52.77273732]
 [ -0.         -52.77273729  -0.        ]
 [-52.77273729  -0.          -0.        ]]


In [10]:
# Now we need to make the ensemble
ensemble = NUTSCanonicalEnsemble(atoms2, temperature=1000,
                                 verbose=True, escape_level=8)

[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
trajectory file None
trying step size 1.0
optimal step size 1.0
thermal_nrg 0.0861738569226
kinetic energy 0.775564712303


In [11]:
# Now run the simulation
traj, metadata = ensemble.run(20)

iteration number 0
	time step size 10.1805069739 fs
	 	depth 1 samples 2
	 	depth 2 samples 4
iteration number 1
	time step size 101.805069739 fs
	 	depth 1 samples 2
iteration number 2
	time step size 24.1632408217 fs
	 	depth 1 samples 2
	 	depth 2 samples 4
	 	depth 3 samples 8
iteration number 3
	time step size 3.40938267521 fs
			New Potential Energy: 77.1178931441 eV
			New Kinetic Energy: 11.4945074096 eV
			New Temperature: 14820.8231076 K
accepted configuration at  77.1178931441
	 	depth 1 samples 2
			New Potential Energy: 81.0621260014 eV
			New Kinetic Energy: 8.70906708994 eV
			New Temperature: 11229.323552 K
accepted configuration at  81.0621260014
	 	depth 2 samples 4
iteration number 4
	time step size 5.56119179605 fs
			New Potential Energy: 44.3825612387 eV
			New Kinetic Energy: 4.66243394282 eV
			New Temperature: 6011.66333236 K
accepted configuration at  44.3825612387
	 	depth 1 samples 2
iteration number 5
	time step size 12.253914273 fs
	 	depth 1 samples 2
	 	

In [ ]:
view(traj)